Installation is simple:

```
Pkg.add("Stan")
```

You need to install CmdStan first though. Get it from [here](http://mc-stan.org/users/interfaces/cmdstan). You also need to set the `CMDSTAN_HOME` environment variable, which should point to the folder in which you unpacked `CmdStan`.

This notebook assumes that you have created a `CmdStanDir` variable (mostly likely by setting it in your `~/.juliarc.jl` file) and that it points to the location in which you unpacked CmdStan.

In [1]:
using Stan

# Use the describe function from Mamba.
#
using Mamba

INFO: Recompiling stale cache file /home/colliera/.julia/lib/v0.6/CodecZlib.ji for module CodecZlib.
INFO: Recompiling stale cache file /home/colliera/.julia/lib/v0.6/Gadfly.ji for module Gadfly.
INFO: Recompiling stale cache file /home/colliera/.julia/lib/v0.6/LightGraphs.ji for module LightGraphs.


Check that environment variable is set.

In [2]:
ENV["CMDSTAN_HOME"]

"/opt/cmdstan-2.17.1/"

## Load Data

In [3]:
using RData

INFO: Recompiling stale cache file /home/colliera/.julia/lib/v0.6/RData.ji for module RData.


In [4]:
pwd()

"/home/colliera/proj/Z-317-talk-first-steps-stan/julia"

In [9]:
slots = load("/home/colliera/proj/Z-317-talk-first-steps-stan/maud-sessions.rda", convert=true)
slots = slots["sessions_stats"];

In [10]:
size(slots, 1)

100

In [11]:
names(slots)

7-element Array{Symbol,1}:
 :session 
 :spins   
 :hits    
 :wager   
 :payout  
 :hit_rate
 :rtp     

In [12]:
slots[1:6,:]

,session,spins,hits,wager,payout,hit_rate,rtp
1,1,7,2,10,3.0,0.285714,0.3
2,2,19,7,30,29.0,0.368421,0.966667
3,3,19,3,22,3.0,0.157895,0.136364
4,4,26,7,30,13.0,0.269231,0.433333
5,5,23,8,31,35.0,0.347826,1.12903
6,6,20,8,26,12.0,0.4,0.461538


Define a model.

In [13]:
const binomial_model = "
data {
  int<lower=0> N;
  int hits[N];
  int spins[N];
}
parameters {
  real<lower=0,upper=1> theta;
}
model {
  hits ~ binomial(spins, theta);       // Likelihood
  theta ~ beta(2, 2);                  // Prior
}
";

Create a `Stanmodel` object. This specifies the basic attributes for the model.

In [24]:
stanmodel = Stanmodel(model=binomial_model, nchains=4);


File /home/colliera/proj/Z-317-talk-first-steps-stan/julia/tmp/binomial.stan will be updated.



That will actually take the model string and write it to a temporary external file with `.stan` extension. The `name` argument can be used to specify the name of this file.

Take a look at the model object.

In [16]:
stanmodel |> display

  name =                    "bernoulli"
  nchains =                 4
  num_samples =             1000
  num_warmup =                   1000
  thin =                    1
  useMamba =                true
  mambaThinning =           1
  monitors =                String[]
  model_file =              "bernoulli.stan"
  data_file =               ""
  output =                  Output()
    file =                    ""
    diagnostics_file =        ""
    refresh =                 100
  method =                  Sample()
    num_samples =             1000
    num_warmup =              1000
    save_warmup =             false
    thin =                    1
    algorithm =               HMC()
      engine =                  NUTS()
        max_depth =               10
      metric =                  

Stan.diag_e
      stepsize =                1.0
      stepsize_jitter =         1.0
    adapt =                   Adapt()
      gamma =                   0.05
      delta =                   0.8
      kappa =                   0.75
      t0 =                      10.0
      init_buffer =             75
      term_buffer =             50
      window =                  25


Interrogate attributes of the model.

In [17]:
stanmodel.method

  method =                  Sample()
    num_samples =             1000
    num_warmup =              1000
    save_warmup =             false
    thin =                    1
    algorithm =               HMC()
      engine =                  NUTS()
        max_depth =               10
      metric =                  Stan.diag_e
      stepsize =                1.0
      stepsize_jitter =         1.0
    adapt =                   Adapt()
      gamma =                   0.05
      delta =                   0.8
      kappa =                   0.75
      t0 =                      10.0
      init_buffer =             75
      term_buffer =             50
      window =                  25


Update characteristics of model. The `Stanmodel` object can be tailored either via the constructor or after the fact.

In [18]:
stanmodel.method.adapt.delta = 0.85

0.85

Set up the observed input data.

In [19]:
const binomial_data = Dict("N"     => size(slots, 1),
                           "hits"  => slots[:hits],
                           "spins" => slots[:spins])

Dict{String,Any} with 3 entries:
  "hits"  => Int32[2, 7, 3, 7, 8, 8, 5, 4, 4, 8  …  8, 5, 2, 2, 10, 8, 3, 5, 4,…
  "N"     => 100
  "spins" => Int32[7, 19, 19, 26, 23, 20, 22, 22, 18, 26  …  19, 16, 14, 18, 21…

Build the code and run the simulation. Uses `make` to recompile code, but only if necessary. Multiple chains are run in parallel (if possible).

In [21]:
rc, simulated = stan(stanmodel, [binomial_data]);



--- Translating Stan model to C++ code ---
bin/stanc  /home/colliera/proj/Z-317-talk-first-steps-stan/julia/tmp/bernoulli.stan --o=/home/colliera/proj/Z-317-talk-first-steps-stan/julia/tmp/bernoulli.hpp
Model name=bernoulli_model
Input file=/home/colliera/proj/Z-317-talk-first-steps-stan/julia/tmp/bernoulli.stan
Output file=/home/colliera/proj/Z-317-talk-first-steps-stan/julia/tmp/bernoulli.hpp

--- Linking C++ model ---
g++ -Wall -I . -isystem stan/lib/stan_math/lib/eigen_3.3.3 -isystem stan/lib/stan_math/lib/boost_1.64.0 -isystem stan/lib/stan_math/lib/cvodes_2.9.0/include -std=c++1y -DBOOST_RESULT_OF_USE_TR1 -DBOOST_NO_DECLTYPE -DBOOST_DISABLE_ASSERTS -DBOOST_PHOENIX_NO_VARIADIC_EXPRESSION -Wno-unused-function -Wno-uninitialized -I src -isystem stan/src -isystem stan/lib/stan_math/ -DFUSION_MAX_VECTOR_SIZE=12 -Wno-unused-local-typedefs -DEIGEN_NO_DEBUG -DNO_FPRINTF_OUTPUT -pipe    -O3 -o /home/colliera/proj/Z-317-talk-first-steps-stan/julia/tmp/bernoulli src/cmdstan/main.cpp -incl

In [ ]:
stanmodel.command

Check for success and examine results. The `simulated` object is of type `Mamba.Chains`.

In [22]:
if rc == 0
  sim = simulated[1:1000, ["lp__", "theta", "accept_stat__"], :]
  describe(sim)
end

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
                   Mean          SD        Naive SE       MCSE      ESS
         lp__ -1228.9554500 0.725633574 0.01147327420 0.0193144041 1000
        theta     0.3127153 0.010491966 0.00016589255 0.0002524322 1000
accept_stat__     0.9442633 0.083037968 0.00131294555 0.0020614201 1000

Quantiles:
                   2.5%           25.0%         50.0%          75.0%          97.5%    
         lp__ -1231.02100000 -1229.12000000 -1228.6800000 -1228.50000000 -1228.45000000
        theta     0.29233515     0.30573575     0.3130270     0.31995125     0.33264253
accept_stat__     0.70115673     0.92222900     0.9786485     0.99946275     1.00000000



In [ ]:
simulated.stan_summary()

In [ ]:
println("Brooks, Gelman and Rubin Convergence Diagnostic")
try
  gelmandiag(sim, mpsrf=true, transform=true) |> display
catch e
  #println(e)
  gelmandiag(sim, mpsrf=false, transform=true) |> display
end

In [ ]:
println("Geweke Convergence Diagnostic")
gewekediag(sim) |> display

In [ ]:
println("Highest Posterior Density Intervals")
hpd(sim) |> display

In [ ]:
println("Cross-Correlations")
cor(sim) |> display

In [ ]:
println("Lag-Autocorrelations")
autocor(sim) |> display

In [ ]:
p = plot(sim, [:trace, :mean, :density, :autocor], legend=true);
draw(p, ncol=4, filename="summaryplot", fmt=:svg)
draw(p, ncol=4, filename="summaryplot", fmt=:pdf)